In [5]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

import matplotlib
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time
import dask.array as da
import mne
import numpy as np
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm.notebook import tqdm

In [6]:

tasks = ["task-med1breath_eeg",
"task-med2_eeg",
"task-think1_eeg",
"task-think2_eeg"]

eegDir = "D:\RnnMachineLearning\eegData"


Create the [id,epoch10sec,MeditationState(Boolean)] data

In [15]:

for i in tqdm(range(1,6)):
    listData =[]
    if i<10:
        sub = "00"+str(i)
    else:
        sub= "0"+str(i)

    for task in tqdm(tasks):
        fileName = f"sub-{sub}_{task}.bdf"
        medState = "med" in fileName
        print("File being read:",fileName)
        ex = mne.io.read_raw_bdf(os.path.join(eegDir,fileName),preload=True)

        if ex.info.get("nchan")==80:
            ex.drop_channels(('GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'Status'))
        else:
            ex.drop_channels(('Status'))
        Epochs = mne.make_fixed_length_epochs(ex,duration=10,verbose=False)
        arrEpochs = Epochs.get_data()
        print(arrEpochs.shape)
        if arrEpochs.shape[2]==20480:
            print("Wrong size:",20480)
            continue

        for i in arrEpochs:

            listData.append([sub,i,medState])
    da.to_npy_stack(f"D:/RnnMachineLearning/stack2/sub-{sub}/",da.from_array(np.array(listData,dtype="object"),chunks=(len(listData))), axis=0)
    # np.save(f"D:\RnnMachineLearning\stackData\sub-{sub}.npy",np.array(listData))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

File being read: sub-001_task-med1breath_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-001_task-med1breath_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 620543  =      0.000 ...   605.999 secs...
Using data from preloaded Raw for 60 events and 10240 original time points ...
0 bad epochs dropped
(60, 72, 10240)
File being read: sub-001_task-med2_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-001_task-med2_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 618495  =      0.000 ...   603.999 secs...
Using data from preloaded Raw for 60 events and 10240 original time points ...
0 bad epochs dropped
(60, 72, 10240)
File being read: sub-001_task-think1_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-001_task-think1_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info str

  0%|          | 0/4 [00:00<?, ?it/s]

File being read: sub-002_task-med1breath_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-002_task-med1breath_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 620543  =      0.000 ...   605.999 secs...
Using data from preloaded Raw for 60 events and 10240 original time points ...
0 bad epochs dropped
(60, 72, 10240)
File being read: sub-002_task-med2_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-002_task-med2_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 619519  =      0.000 ...   604.999 secs...
Using data from preloaded Raw for 60 events and 10240 original time points ...
0 bad epochs dropped
(60, 72, 10240)
File being read: sub-002_task-think1_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-002_task-think1_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info str

...

File being read: sub-003_task-med1breath_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-003_task-med1breath_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 616447  =      0.000 ...   601.999 secs...
Using data from preloaded Raw for 60 events and 10240 original time points ...
0 bad epochs dropped
(60, 72, 10240)
File being read: sub-003_task-med2_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-003_task-med2_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 617471  =      0.000 ...   602.999 secs...
Using data from preloaded Raw for 60 events and 10240 original time points ...
0 bad epochs dropped
(60, 72, 10240)
File being read: sub-003_task-think1_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-003_task-think1_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info str

  0%|          | 0/4 [00:00<?, ?it/s]

File being read: sub-004_task-med1breath_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-004_task-med1breath_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 630783  =      0.000 ...   615.999 secs...
Using data from preloaded Raw for 61 events and 10240 original time points ...
0 bad epochs dropped
(61, 72, 10240)
File being read: sub-004_task-med2_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-004_task-med2_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 627711  =      0.000 ...   612.999 secs...
Using data from preloaded Raw for 61 events and 10240 original time points ...
0 bad epochs dropped
(61, 72, 10240)
File being read: sub-004_task-think1_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-004_task-think1_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info str

  0%|          | 0/4 [00:00<?, ?it/s]

File being read: sub-005_task-med1breath_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-005_task-med1breath_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 616447  =      0.000 ...   601.999 secs...
Using data from preloaded Raw for 60 events and 10240 original time points ...
0 bad epochs dropped
(60, 72, 10240)
File being read: sub-005_task-med2_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-005_task-med2_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 672767  =      0.000 ...   656.999 secs...
Using data from preloaded Raw for 65 events and 10240 original time points ...
0 bad epochs dropped
(65, 72, 10240)
File being read: sub-005_task-think1_eeg.bdf
Extracting EDF parameters from D:\RnnMachineLearning\eegData\sub-005_task-think1_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info str

array(['001',
       array([[-0.25932563, -0.25932616, -0.25932607, ..., -0.25932588,
               -0.25932538, -0.25932538],
              [-0.00743725, -0.00743928, -0.00744091, ..., -0.00736331,
               -0.00736335, -0.0073655 ],
              [-0.00560863, -0.00561094, -0.00561194, ..., -0.00560435,
               -0.00560297, -0.00560416],
              ...,
              [ 0.0039566 ,  0.0039587 ,  0.00395973, ...,  0.00395079,
                0.00395463,  0.00395432],
              [-0.00658553, -0.00658869, -0.00659407, ..., -0.006515  ,
               -0.00656972, -0.00663257],
              [ 0.00475207,  0.00474994,  0.00474923, ...,  0.00480979,
                0.00481107,  0.00480794]])                             ,
       True], dtype=object)

In [4]:
arrData = np.array(listData,dtype=object)

np.save("D:\RnnMachineLearning\meditation_data.npy",arrData)
arrData.shape

(1091, 3)

In [ ]:
filterArrtrain = []
filterArrtest = []
for i in arrData:
    if i[0] in ("001","002","003"):
        filterArrtrain.append(True)
        filterArrtest.append(False)
    else:
        filterArrtest.append(True)
        filterArrtrain.append(False)

arrtrain = arrData[filterArrtrain]
arrtest = arrData[filterArrtest]

In [77]:
class EegDataSet(Dataset):

	def __init__(self,dataArray):
		#data loading
		self.x = torch.from_numpy(np.stack(dataArray[:,1]))
		self.labels = torch.from_numpy(dataArray[:,2].astype("float32"))

	def __len__(self):
		# len(dataset)
		return len(self.labels)

	def __getitem__(self, idx):
		return self.x[idx], self.labels[idx]



In [78]:
dataArrTrain = EegDataSet(arrtrain)
dataArrTest = EegDataSet(arrtest)
batch_size=20
dlDataArrTrain = DataLoader(dataArrTrain, batch_size=batch_size, shuffle=True)
dlDataArrtest = DataLoader(dataArrTest, batch_size=batch_size, shuffle=True)

In [79]:
# for x,y in dlDataArr:
# 	print(torch.reshape(x,(x.shape[0],1,x.shape[1],x.shape[2])).shape)

#[batch_size,1,80,10240]

In [80]:
class Cnn_eeg(Module):
    def __init__(self,batch_size):
        super().__init__()
        self.batch_size=batch_size
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(self.linear_input_neurons(), 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # print("0:",x.shape)
        x = torch.reshape(x,(x.shape[0],1,x.shape[1],x.shape[2]))
        # print("1:",x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        # print("2:",x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        # print("3:",x.shape)
        x = x.view(x.shape[0], -1) # flatten all dimensions except batch
        # print("4:",x.shape)
        x = F.relu(self.fc1(x))
        # print("5:",x.shape)
        x = F.relu(self.fc2(x))
        # print("6:",x.shape)
        x = self.fc3(x)
        # print("7:",x.shape)
        return x
        # here we apply convolution operations before linear layer, and it returns the 4-dimensional size tensor.
    def size_after_relu(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        return x.size()[1:]


    # after obtaining the size in above method, we call it and multiply all elements of the returned size.
    def linear_input_neurons(self):
        size = self.size_after_relu(torch.rand(self.batch_size, 1, 72, 10240))
        m = 1
        for i in size:
            m *= i

        return int(m)

In [81]:
net = Cnn_eeg(batch_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [82]:
for epoch in range(3):  # loop over the dataset multiple times


    for i, data in tqdm(enumerate(dlDataArrTrain, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(torch.float32),data[1].to(torch.int64)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # print(outputs)
        # print(outputs.shape)
        # print(outputs.dtype)
        # print(labels.shape)
        # print(labels.dtype)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        print("batch:",i)
        print("Loss:",loss)



print('Finished Training')

0it [00:00, ?it/s]

batch: 0
Loss: tensor(0.7051, grad_fn=<NllLossBackward0>)
batch: 1
Loss: tensor(5.4619, grad_fn=<NllLossBackward0>)
batch: 2
Loss: tensor(2.2543, grad_fn=<NllLossBackward0>)
batch: 3
Loss: tensor(0.7488, grad_fn=<NllLossBackward0>)
batch: 4
Loss: tensor(0.7009, grad_fn=<NllLossBackward0>)
batch: 5
Loss: tensor(0.7238, grad_fn=<NllLossBackward0>)
batch: 6
Loss: tensor(0.7252, grad_fn=<NllLossBackward0>)
batch: 7
Loss: tensor(0.6475, grad_fn=<NllLossBackward0>)
batch: 8
Loss: tensor(0.7705, grad_fn=<NllLossBackward0>)
batch: 9
Loss: tensor(0.6962, grad_fn=<NllLossBackward0>)
batch: 10
Loss: tensor(0.6768, grad_fn=<NllLossBackward0>)
batch: 11
Loss: tensor(0.7677, grad_fn=<NllLossBackward0>)
batch: 12
Loss: tensor(0.7033, grad_fn=<NllLossBackward0>)
batch: 13
Loss: tensor(0.7657, grad_fn=<NllLossBackward0>)
batch: 14
Loss: tensor(0.6901, grad_fn=<NllLossBackward0>)
batch: 15
Loss: tensor(0.8596, grad_fn=<NllLossBackward0>)
batch: 16
Loss: tensor(0.6771, grad_fn=<NllLossBackward0>)
batch: 

0it [00:00, ?it/s]

batch: 0
Loss: tensor(0.6928, grad_fn=<NllLossBackward0>)
batch: 1
Loss: tensor(0.7723, grad_fn=<NllLossBackward0>)
batch: 2
Loss: tensor(0.6945, grad_fn=<NllLossBackward0>)
batch: 3
Loss: tensor(0.6886, grad_fn=<NllLossBackward0>)
batch: 4
Loss: tensor(0.6742, grad_fn=<NllLossBackward0>)
batch: 5
Loss: tensor(0.6835, grad_fn=<NllLossBackward0>)
batch: 6
Loss: tensor(0.7257, grad_fn=<NllLossBackward0>)
batch: 7
Loss: tensor(0.7117, grad_fn=<NllLossBackward0>)
batch: 8
Loss: tensor(0.6700, grad_fn=<NllLossBackward0>)
batch: 9
Loss: tensor(0.6811, grad_fn=<NllLossBackward0>)
batch: 10
Loss: tensor(0.6592, grad_fn=<NllLossBackward0>)
batch: 11
Loss: tensor(0.6620, grad_fn=<NllLossBackward0>)
batch: 12
Loss: tensor(0.6370, grad_fn=<NllLossBackward0>)
batch: 13
Loss: tensor(0.6542, grad_fn=<NllLossBackward0>)
batch: 14
Loss: tensor(0.6747, grad_fn=<NllLossBackward0>)
batch: 15
Loss: tensor(0.7096, grad_fn=<NllLossBackward0>)
batch: 16
Loss: tensor(0.6289, grad_fn=<NllLossBackward0>)
batch: 

0it [00:00, ?it/s]

batch: 0
Loss: tensor(0.5437, grad_fn=<NllLossBackward0>)
batch: 1
Loss: tensor(0.5357, grad_fn=<NllLossBackward0>)
batch: 2
Loss: tensor(0.4336, grad_fn=<NllLossBackward0>)
batch: 3
Loss: tensor(0.4952, grad_fn=<NllLossBackward0>)
batch: 4
Loss: tensor(0.4601, grad_fn=<NllLossBackward0>)
batch: 5
Loss: tensor(0.5368, grad_fn=<NllLossBackward0>)
batch: 6
Loss: tensor(0.4870, grad_fn=<NllLossBackward0>)
batch: 7
Loss: tensor(0.6968, grad_fn=<NllLossBackward0>)
batch: 8
Loss: tensor(0.7188, grad_fn=<NllLossBackward0>)
batch: 9
Loss: tensor(0.5063, grad_fn=<NllLossBackward0>)
batch: 10
Loss: tensor(0.4102, grad_fn=<NllLossBackward0>)
batch: 11
Loss: tensor(0.4460, grad_fn=<NllLossBackward0>)
batch: 12
Loss: tensor(0.3660, grad_fn=<NllLossBackward0>)
batch: 13
Loss: tensor(0.7366, grad_fn=<NllLossBackward0>)
batch: 14
Loss: tensor(0.4685, grad_fn=<NllLossBackward0>)
batch: 15
Loss: tensor(0.6097, grad_fn=<NllLossBackward0>)
batch: 16
Loss: tensor(0.4730, grad_fn=<NllLossBackward0>)
batch: 

In [83]:
losses = []
for i, data in tqdm(enumerate(dlDataArrtest, 0)):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data[0].to(torch.float32),data[1].to(torch.int64)



    # forward + backward + optimize
    outputs = net(inputs)
    # print(outputs)
    # print(outputs.shape)
    # print(outputs.dtype)
    # print(labels.shape)
    # print(labels.dtype)
    loss = criterion(outputs, labels)
    losses.append(loss.item())

    # print statistics
    print("batch:",i)
    print("Loss:",loss)

0it [00:00, ?it/s]

batch: 0
Loss: tensor(0.3868, grad_fn=<NllLossBackward0>)
batch: 1
Loss: tensor(0.3688, grad_fn=<NllLossBackward0>)
batch: 2
Loss: tensor(0.4335, grad_fn=<NllLossBackward0>)
batch: 3
Loss: tensor(0.4581, grad_fn=<NllLossBackward0>)
batch: 4
Loss: tensor(0.4408, grad_fn=<NllLossBackward0>)
batch: 5
Loss: tensor(0.5864, grad_fn=<NllLossBackward0>)
batch: 6
Loss: tensor(0.4602, grad_fn=<NllLossBackward0>)
batch: 7
Loss: tensor(0.4249, grad_fn=<NllLossBackward0>)
batch: 8
Loss: tensor(0.4342, grad_fn=<NllLossBackward0>)
batch: 9
Loss: tensor(0.4479, grad_fn=<NllLossBackward0>)
batch: 10
Loss: tensor(0.4878, grad_fn=<NllLossBackward0>)
batch: 11
Loss: tensor(0.4780, grad_fn=<NllLossBackward0>)
batch: 12
Loss: tensor(0.4696, grad_fn=<NllLossBackward0>)
batch: 13
Loss: tensor(0.5608, grad_fn=<NllLossBackward0>)
batch: 14
Loss: tensor(0.3705, grad_fn=<NllLossBackward0>)
batch: 15
Loss: tensor(0.4265, grad_fn=<NllLossBackward0>)
batch: 16
Loss: tensor(0.3796, grad_fn=<NllLossBackward0>)
batch: 

In [84]:
# switch off autograd for evaluation
lossFinale= sum(losses)/len(losses)

In [85]:
lossFinale

0.4491594888662037